In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from keras.layers import Dense,Input,Lambda
from keras.models import Model
from keras.metrics import binary_crossentropy
import keras.backend as K
from keras.datasets import mnist
from keras.utils import to_categorical

from scipy.stats import norm
from sklearn.manifold import Isomap

Using TensorFlow backend.
/Users/elijahc/.pyenv/versions/3.5.3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:
from src.utils import process_mnist,gen_trajectory,gen_sorted_isomap
from src.data_loader import prepare_keras_dataset

In [3]:
# train the VAE on MNIST digits
(x_train,y_train),(x_test,y_test) = prepare_keras_dataset(mnist)

In [4]:
# plt.imshow(x_train[3].reshape(28,28))

In [5]:
from src.config import get_config
from src.models import VAE_Builder
conf, unparsed_args = get_config()

In [6]:
mod_builder = VAE_Builder()

In [7]:
mod_builder.build(input_shape=(784,))

NameError: name 'latent_dim' is not defined

In [ ]:
encoding_dim = 64
latent_dim = 2
inputs = Input(shape=input_shape)
encoded = build_dense(inputs,[512,encoding_dim],activations='relu')

In [ ]:
z_mean = Dense(latent_dim)(encoded)
z_log_sigma = Dense(latent_dim)(encoded)

def sampler(args):
    mean,log_stddev = args
    std_norm = K.random_normal(shape=(K.shape(mean)[0],latent_dim),mean=0,stddev=1)
    
    return mean + K.exp(log_stddev) * std_norm

lat_vec = Lambda(sampler)([z_mean,z_log_sigma])

In [ ]:
decoded_mean = build_dense(lat_vec,[encoding_dim,256,784],activations=['relu','relu','sigmoid'])

In [ ]:
vae = Model(inputs,decoded_mean)
def vae_loss(input_img,output):
    # Reconstruction loss
    reconstruction_loss = K.sum(K.square(output-input_img))
    kl_loss = -0.5*K.sum(1+(2*z_log_sigma)-K.square(z_mean)- K.square(K.exp(z_log_sigma)),axis=-1)
    total_loss = K.mean(reconstruction_loss)+kl_loss
    return total_loss
# vae.compile(loss=vae_loss,optimizer='rmsprop')
vae.compile(loss=vae_loss,optimizer='adadelta')

In [ ]:
vae.summary()

In [ ]:
vae.fit(x_train, x_train,
        shuffle=True,
        epochs=50,
        batch_size=128,
        validation_data=(x_test, x_test))

In [ ]:
encoder = Model(inputs,z_mean)
decoder_inp = Input(shape=(2,))
# _generator_x = build_dense(decoder_inp,[encoding_dim,256,784],activations=['relu','relu','sigmoid'])
# generator = Model(decoder_inp,decoded_mean)
# print(generator.summary())
dec_layers = vae.layers[-3:]
_gen_x = dec_layers[0](decoder_inp)
_gen_x = dec_layers[1](_gen_x)
outputs = dec_layers[2](_gen_x)
generator = Model(decoder_inp,outputs)

In [ ]:
x_test_encoded = encoder.predict(x_test,batch_size=128)

In [ ]:
plt.imshow(x_test[2].reshape(28,28))
# generator.predict()

In [ ]:
# x_test_encoded[2]
dec_test = generator.predict(x_test_encoded[:5])
plt.imshow(dec_test[2].reshape(28,28))

In [ ]:
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
plt.colorbar()
plt.show()

In [ ]:
encoder.summary()

In [ ]:
from datetime import date
print(date.today())
save_dir = '/home/elijahc/projects/vae/models/'+str(date.today())+'/'


In [ ]:
# vae.save(save_dir+'vae_3layer.h5',include_optimizer=False)
# encoder.save(save_dir+'enc.h5',include_optimizer=False)

In [ ]:
x0 = np.array([0,-5])
x1 = np.array([0,5])
traj = gen_trajectory(x0,x1,delta=.1)
# traj[:3].shape
dec_traj = K.get_value(generator(K.variable(traj)))
dec_traj = dec_traj.reshape(11,28,28)
dec_traj.shape

In [ ]:
fig, axs = plt.subplots(1,11,figsize=(10,10))
for i,ax in enumerate(axs):
    
    ax.imshow(dec_traj[i])

In [ ]:
enc_32 = vae.layers[6]
enc_256 = vae.layers[7]

In [ ]:
encoder.trainable=False
x = enc_32(encoder.outputs[0])
y_class_oh = Dense(10,activation='softmax')(x)

In [ ]:
med = Model(inputs=inputs,outputs=y_class_oh)
med.layers[-2].trainable=False
for l in med.layers[1:4]:
    l.trainable=False
med.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
y_train_oh = to_categorical(y_train,num_classes=10)

In [ ]:
y_train_oh.shape

In [ ]:
med.summary()

In [ ]:
med.fit(x_train,y_train_oh,
        batch_size=128,
        epochs=25,
        validation_data=(x_test,to_categorical(y_test,num_classes=10))
       )

In [ ]:
x_g = generator.predict(x_test_encoded[:3])
y_test_im = x_g.reshape(3,28,28)

In [ ]:
x_test_im = x_test.reshape(10000,28,28)[:3]

fig,axs = plt.subplots(1,3)
for im,ax in zip(x_test_im,axs):
    ax.imshow(im)

In [ ]:
fig,axs = plt.subplots(1,3)
for im,ax in zip(y_test_im,axs):
    ax.imshow(im)

In [ ]:
med.evaluate(x_test,to_categorical(y_test,num_classes=10))